In [27]:
from pyspark.sql import SparkSession
from operator import add

spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.250:7077") \
        .appName("AlmaLundberg_Sparkapplication")\
        .config("spark.dynamicAllocation.enabled", False)\
        .config("spark.cores.max", 4)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

# RDD API
spark_context = spark_session.sparkContext

spark_context.setLogLevel("INFO")

24/02/26 20:58:29 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/02/26 20:58:29 INFO SharedState: Warehouse path is 'file:/home/ubuntu/dataengineering_AlmaLundberg/A3/spark-warehouse'.
24/02/26 20:58:30 INFO BlockManagerInfo: Removed broadcast_18_piece0 on 192.168.2.252:10005 in memory (size: 8.8 KiB, free: 434.3 MiB)
24/02/26 20:58:30 INFO BlockManagerInfo: Removed broadcast_18_piece0 on 192.168.2.250:10005 in memory (size: 8.8 KiB, free: 434.3 MiB)
24/02/26 20:58:30 INFO BlockManagerInfo: Removed broadcast_18_piece0 on host-192-168-2-136-de1:10005 in memory (size: 8.8 KiB, free: 434.3 MiB)


In [28]:
# Using map and reduce from the Spark API, and loading the text file from HDFS.

lines_en = spark_context.textFile("hdfs://192.168.2.250:9000/europarl/europarl-v7.sv-en.en")
num_lines_en = lines_en.count()
print(f"Total lines in English file: {num_lines_en}") 

lines_sv = spark_context.textFile("hdfs://192.168.2.250:9000/europarl/europarl-v7.sv-en.sv")
num_lines_sv = lines_sv.count()
print(f"Total lines in Swedish file: {num_lines_sv}")


# print(lines.first())
# words = lines.map(lambda line: line.split(' '))
# word_counts = words.map(lambda w: len(w))
# total_words = word_counts.reduce(add)
# print(f'total words= {total_words}') 

#lines = spark_context.textFile("/home/ubuntu/i_have_a_dream.txt")
# print(lines.first())
# words = lines.map(lambda line: line.split(' '))
# word_counts = words.map(lambda w: len(w))
# total_words = word_counts.reduce(add)
# print(f'total words= {total_words}')  
# ... the same number of words?

24/02/26 20:58:32 INFO MemoryStore: Block broadcast_19 stored as values in memory (estimated size 221.7 KiB, free 433.7 MiB)
24/02/26 20:58:32 INFO MemoryStore: Block broadcast_19_piece0 stored as bytes in memory (estimated size 32.7 KiB, free 433.7 MiB)
24/02/26 20:58:32 INFO BlockManagerInfo: Added broadcast_19_piece0 in memory on host-192-168-2-136-de1:10005 (size: 32.7 KiB, free: 434.3 MiB)
24/02/26 20:58:32 INFO SparkContext: Created broadcast 19 from textFile at NativeMethodAccessorImpl.java:0
24/02/26 20:58:32 INFO FileInputFormat: Total input files to process : 1
24/02/26 20:58:32 INFO NetworkTopology: Adding a new node: /default-rack/192.168.2.250:9866
24/02/26 20:58:32 INFO SparkContext: Starting job: count at /tmp/ipykernel_31804/3631255496.py:4
24/02/26 20:58:32 INFO DAGScheduler: Got job 15 (count at /tmp/ipykernel_31804/3631255496.py:4) with 2 output partitions
24/02/26 20:58:32 INFO DAGScheduler: Final stage: ResultStage 21 (count at /tmp/ipykernel_31804/3631255496.py:4)

Total lines in English file: 1862234


24/02/26 20:58:37 INFO TaskSetManager: Finished task 2.0 in stage 22.0 (TID 80) in 236 ms on 192.168.2.250 (executor 1) (1/3)


Total lines in Swedish file: 1862234


24/02/26 20:58:38 INFO TaskSetManager: Finished task 1.0 in stage 22.0 (TID 79) in 1098 ms on 192.168.2.252 (executor 0) (2/3)
24/02/26 20:58:38 INFO TaskSetManager: Finished task 0.0 in stage 22.0 (TID 78) in 1281 ms on 192.168.2.250 (executor 1) (3/3)
24/02/26 20:58:38 INFO TaskSchedulerImpl: Removed TaskSet 22.0, whose tasks have all completed, from pool 
24/02/26 20:58:38 INFO DAGScheduler: ResultStage 22 (count at /tmp/ipykernel_31804/3631255496.py:8) finished in 1.295 s
24/02/26 20:58:38 INFO DAGScheduler: Job 16 is finished. Cancelling potential speculative or zombie tasks for this job
24/02/26 20:58:38 INFO TaskSchedulerImpl: Killing all running tasks in stage 22: Stage finished
24/02/26 20:58:38 INFO DAGScheduler: Job 16 finished: count at /tmp/ipykernel_31804/3631255496.py:8, took 1.298727 s


In [29]:
# Total lines in English file: 1862234
# Total lines in Swedish file: 1862234

In [36]:
# Counting nmr of partitions in the English file
num_partitions_en = lines_en.getNumPartitions()
print(f"Number of partitions in English file: {num_partitions_en}")

# Counting nmr of partitions in the Swedish file
num_partitions_sv = lines_sv.getNumPartitions()
print(f"Number of partitions in Swedish file: {num_partitions_sv}")

Number of partitions in English file: 2
Number of partitions in Swedish file: 3


In [37]:
# Number of partitions in English file: 2
# Number of partitions in Swedish file: 3

In [79]:
def preprocess_text(line):
    return line.lower().split(' ')

In [80]:
processed_lines_en = lines_en.map(preprocess_text)
processed_lines_sv = lines_sv.map(preprocess_text)

In [81]:
# Verify that the line counts still match after the pre-processing:

processed_count_en = processed_lines_en.count()
processed_count_sv = processed_lines_sv.count()
print(f"Processed line count in English: {processed_count_en}")
print(f"Processed line count in Swedish: {processed_count_sv}")

24/02/26 21:42:04 INFO SparkContext: Starting job: count at /tmp/ipykernel_31804/226898882.py:3
24/02/26 21:42:04 INFO DAGScheduler: Got job 46 (count at /tmp/ipykernel_31804/226898882.py:3) with 2 output partitions
24/02/26 21:42:04 INFO DAGScheduler: Final stage: ResultStage 63 (count at /tmp/ipykernel_31804/226898882.py:3)
24/02/26 21:42:04 INFO DAGScheduler: Parents of final stage: List()
24/02/26 21:42:04 INFO DAGScheduler: Missing parents: List()
24/02/26 21:42:04 INFO DAGScheduler: Submitting ResultStage 63 (PythonRDD[104] at count at /tmp/ipykernel_31804/226898882.py:3), which has no missing parents
24/02/26 21:42:04 INFO MemoryStore: Block broadcast_59 stored as values in memory (estimated size 9.6 KiB, free 433.4 MiB)
24/02/26 21:42:04 INFO BlockManagerInfo: Removed broadcast_58_piece0 on host-192-168-2-136-de1:10005 in memory (size: 6.5 KiB, free: 434.3 MiB)
24/02/26 21:42:04 INFO MemoryStore: Block broadcast_59_piece0 stored as bytes in memory (estimated size 5.7 KiB, free 

Processed line count in English: 1862234
Processed line count in Swedish: 1862234


24/02/26 21:42:12 INFO TaskSetManager: Finished task 1.0 in stage 64.0 (TID 166) in 2837 ms on 192.168.2.252 (executor 0) (2/3)
24/02/26 21:42:12 INFO TaskSetManager: Finished task 0.0 in stage 64.0 (TID 165) in 2840 ms on 192.168.2.250 (executor 1) (3/3)
24/02/26 21:42:12 INFO TaskSchedulerImpl: Removed TaskSet 64.0, whose tasks have all completed, from pool 
24/02/26 21:42:12 INFO DAGScheduler: ResultStage 64 (count at /tmp/ipykernel_31804/226898882.py:4) finished in 2.854 s
24/02/26 21:42:12 INFO DAGScheduler: Job 47 is finished. Cancelling potential speculative or zombie tasks for this job
24/02/26 21:42:12 INFO TaskSchedulerImpl: Killing all running tasks in stage 64: Stage finished
24/02/26 21:42:12 INFO DAGScheduler: Job 47 finished: count at /tmp/ipykernel_31804/226898882.py:4, took 2.857187 s


In [82]:
# Processed line count in English: 1862234
# Processed line count in Swedish: 1862234

# The line count still match after pre-processing

In [83]:
# Check 10 most frequent words in English and Swedish

words_en = processed_lines_en.flatMap(lambda line: line)
words_sv = processed_lines_sv.flatMap(lambda line: line)

word_counts_en = words_en.map(lambda word: (word, 1)).reduceByKey(add)
word_counts_sv = words_sv.map(lambda word: (word, 1)).reduceByKey(add)

top_10_words_en = word_counts_en.takeOrdered(10, key=lambda x: -x[1])
top_10_words_sv = word_counts_sv.takeOrdered(10, key=lambda x: -x[1])

print("\nTop 10 words in English:")
for word, count in top_10_words_en:
    print(f'{word}: {count}')
    
print("\nTop 10 words in Swedish:")
for word, count in top_10_words_sv:
    print(f'{word}: {count}')

24/02/26 21:42:40 INFO SparkContext: Starting job: takeOrdered at /tmp/ipykernel_31804/2132416953.py:9
24/02/26 21:42:40 INFO DAGScheduler: Registering RDD 107 (reduceByKey at /tmp/ipykernel_31804/2132416953.py:6) as input to shuffle 11
24/02/26 21:42:40 INFO DAGScheduler: Got job 48 (takeOrdered at /tmp/ipykernel_31804/2132416953.py:9) with 2 output partitions
24/02/26 21:42:40 INFO DAGScheduler: Final stage: ResultStage 66 (takeOrdered at /tmp/ipykernel_31804/2132416953.py:9)
24/02/26 21:42:40 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 65)
24/02/26 21:42:40 INFO DAGScheduler: Missing parents: List(ShuffleMapStage 65)
24/02/26 21:42:40 INFO DAGScheduler: Submitting ShuffleMapStage 65 (PairwiseRDD[107] at reduceByKey at /tmp/ipykernel_31804/2132416953.py:6), which has no missing parents
24/02/26 21:42:40 INFO MemoryStore: Block broadcast_61 stored as values in memory (estimated size 12.9 KiB, free 433.4 MiB)
24/02/26 21:42:40 INFO BlockManagerInfo: Removed broadcas


Top 10 words in English:
the: 3498375
of: 1659758
to: 1539760
and: 1288401
in: 1085993
that: 797516
a: 773522
is: 758050
for: 534242
we: 522849

Top 10 words in Swedish:
att: 1706293
och: 1344830
i: 1050774
det: 924866
som: 913276
för: 908680
av: 738068
är: 694381
en: 620310
vi: 539797


24/02/26 21:43:12 INFO TaskSetManager: Finished task 1.0 in stage 68.0 (TID 176) in 236 ms on 192.168.2.252 (executor 0) (1/3)
24/02/26 21:43:12 INFO TaskSetManager: Finished task 2.0 in stage 68.0 (TID 177) in 260 ms on 192.168.2.250 (executor 1) (2/3)
24/02/26 21:43:12 INFO TaskSetManager: Finished task 0.0 in stage 68.0 (TID 175) in 299 ms on 192.168.2.250 (executor 1) (3/3)
24/02/26 21:43:12 INFO TaskSchedulerImpl: Removed TaskSet 68.0, whose tasks have all completed, from pool 
24/02/26 21:43:12 INFO DAGScheduler: ResultStage 68 (takeOrdered at /tmp/ipykernel_31804/2132416953.py:10) finished in 0.315 s
24/02/26 21:43:12 INFO DAGScheduler: Job 49 is finished. Cancelling potential speculative or zombie tasks for this job
24/02/26 21:43:12 INFO TaskSchedulerImpl: Killing all running tasks in stage 68: Stage finished
24/02/26 21:43:12 INFO DAGScheduler: Job 49 finished: takeOrdered at /tmp/ipykernel_31804/2132416953.py:10, took 14.286866 s


In [84]:
# Top 10 words in English corpus: [('the', 3498375), ('of', 1659758), ('to', 1539760), ('and', 1288401), ('in', 1085993), 
#                                  ('that', 797516), ('a', 773522), ('is', 758050), ('for', 534242), ('we', 522849)]
# Top 10 words in Swedish corpus: [('att', 1706293), ('och', 1344830), ('i', 1050774), ('det', 924866), ('som', 913276), 
#                                  ('för', 908680), ('av', 738068), ('är', 694381), ('en', 620310), ('vi', 539797)]

# OBS!!! KOLLA DETTA SÅ DET STÄMMER, DETTA ÄR OM DET ÄR SPLIT(' ') MEN SAGAS RESULTAT ÄR OM DET ÄR SPLIT() i preprocessing delen
# Top 10 words in English:
# the: 3498375
# of: 1659758
# to: 1539760
# and: 1288401
# in: 1085993
# that: 797516
# a: 773522
# is: 758050
# for: 534242
# we: 522849

# Top 10 words in Swedish:
# att: 1706293
# och: 1344830
# i: 1050774
# det: 924866
# som: 913276
# för: 908680
# av: 738068
# är: 694381
# en: 620310
# vi: 539797

# The results are reasonable, as the most frequent words are common words used to build up sentences (konjunctions, prepositions etc)
# In the Swedish translation for example, the words also match most of the top 10 most frequent words in the Swedish language like i, och, att, det, som, en, är, av, för 

In [85]:
# Key the lines by their line number and swap
sv_swapped = processed_lines_sv.zipWithIndex().map(lambda x: (x[1], x[0]))
en_swapped = processed_lines_en.zipWithIndex().map(lambda x: (x[1], x[0]))

24/02/26 21:43:54 INFO SparkContext: Starting job: zipWithIndex at /tmp/ipykernel_31804/4241438500.py:2
24/02/26 21:43:54 INFO DAGScheduler: Got job 50 (zipWithIndex at /tmp/ipykernel_31804/4241438500.py:2) with 3 output partitions
24/02/26 21:43:54 INFO DAGScheduler: Final stage: ResultStage 69 (zipWithIndex at /tmp/ipykernel_31804/4241438500.py:2)
24/02/26 21:43:54 INFO DAGScheduler: Parents of final stage: List()
24/02/26 21:43:54 INFO DAGScheduler: Missing parents: List()
24/02/26 21:43:54 INFO DAGScheduler: Submitting ResultStage 69 (PythonRDD[116] at zipWithIndex at /tmp/ipykernel_31804/4241438500.py:2), which has no missing parents
24/02/26 21:43:54 INFO MemoryStore: Block broadcast_65 stored as values in memory (estimated size 8.3 KiB, free 433.4 MiB)
24/02/26 21:43:54 INFO MemoryStore: Block broadcast_65_piece0 stored as bytes in memory (estimated size 5.1 KiB, free 433.4 MiB)
24/02/26 21:43:54 INFO BlockManagerInfo: Added broadcast_65_piece0 in memory on host-192-168-2-136-de

In [86]:
# Step 1: Key the lines by their line number
#sv_keyed = processed_lines_sv.zipWithIndex().map(lambda x: (x[1], x[0]))
#en_keyed = processed_lines_en.zipWithIndex().map(lambda x: (x[1], x[0]))

In [87]:
# Swap the key and value
#sv_swapped = sv_with_index.map(lambda x: (x[1], x[0]))
#en_swapped = en_with_index.map(lambda x: (x[1], x[0]))

In [88]:
sv_swapped.take(10)

24/02/26 21:44:31 INFO SparkContext: Starting job: runJob at PythonRDD.scala:181
24/02/26 21:44:31 INFO DAGScheduler: Got job 52 (runJob at PythonRDD.scala:181) with 1 output partitions
24/02/26 21:44:31 INFO DAGScheduler: Final stage: ResultStage 71 (runJob at PythonRDD.scala:181)
24/02/26 21:44:31 INFO DAGScheduler: Parents of final stage: List()
24/02/26 21:44:31 INFO DAGScheduler: Missing parents: List()
24/02/26 21:44:31 INFO DAGScheduler: Submitting ResultStage 71 (PythonRDD[118] at RDD at PythonRDD.scala:53), which has no missing parents
24/02/26 21:44:31 INFO MemoryStore: Block broadcast_67 stored as values in memory (estimated size 9.2 KiB, free 433.4 MiB)
24/02/26 21:44:31 INFO BlockManagerInfo: Removed broadcast_66_piece0 on host-192-168-2-136-de1:10005 in memory (size: 5.1 KiB, free: 434.3 MiB)
24/02/26 21:44:31 INFO BlockManagerInfo: Removed broadcast_66_piece0 on 192.168.2.252:10005 in memory (size: 5.1 KiB, free: 434.3 MiB)
24/02/26 21:44:31 INFO BlockManagerInfo: Remove

[(0, ['återupptagande', 'av', 'sessionen']),
 (1,
  ['jag',
   'förklarar',
   'europaparlamentets',
   'session',
   'återupptagen',
   'efter',
   'avbrottet',
   'den',
   '17',
   'december.',
   'jag',
   'vill',
   'på',
   'nytt',
   'önska',
   'er',
   'ett',
   'gott',
   'nytt',
   'år',
   'och',
   'jag',
   'hoppas',
   'att',
   'ni',
   'haft',
   'en',
   'trevlig',
   'semester.']),
 (2,
  ['som',
   'ni',
   'kunnat',
   'konstatera',
   'ägde',
   '"den',
   'stora',
   'år',
   '2000-buggen"',
   'aldrig',
   'rum.',
   'däremot',
   'har',
   'invånarna',
   'i',
   'ett',
   'antal',
   'av',
   'våra',
   'medlemsländer',
   'drabbats',
   'av',
   'naturkatastrofer',
   'som',
   'verkligen',
   'varit',
   'förskräckliga.']),
 (3,
  ['ni',
   'har',
   'begärt',
   'en',
   'debatt',
   'i',
   'ämnet',
   'under',
   'sammanträdesperiodens',
   'kommande',
   'dagar.']),
 (4,
  ['till',
   'dess',
   'vill',
   'jag',
   'att',
   'vi,',
   'som',
   'ett',
 

In [89]:
en_swapped.take(10)

24/02/26 21:44:59 INFO SparkContext: Starting job: runJob at PythonRDD.scala:181
24/02/26 21:44:59 INFO DAGScheduler: Got job 53 (runJob at PythonRDD.scala:181) with 1 output partitions
24/02/26 21:44:59 INFO DAGScheduler: Final stage: ResultStage 72 (runJob at PythonRDD.scala:181)
24/02/26 21:44:59 INFO DAGScheduler: Parents of final stage: List()
24/02/26 21:44:59 INFO DAGScheduler: Missing parents: List()
24/02/26 21:44:59 INFO DAGScheduler: Submitting ResultStage 72 (PythonRDD[119] at RDD at PythonRDD.scala:53), which has no missing parents
24/02/26 21:44:59 INFO MemoryStore: Block broadcast_68 stored as values in memory (estimated size 9.2 KiB, free 433.4 MiB)
24/02/26 21:44:59 INFO MemoryStore: Block broadcast_68_piece0 stored as bytes in memory (estimated size 5.6 KiB, free 433.4 MiB)
24/02/26 21:44:59 INFO BlockManagerInfo: Added broadcast_68_piece0 in memory on host-192-168-2-136-de1:10005 (size: 5.6 KiB, free: 434.3 MiB)
24/02/26 21:44:59 INFO SparkContext: Created broadcast 

[(0, ['resumption', 'of', 'the', 'session']),
 (1,
  ['i',
   'declare',
   'resumed',
   'the',
   'session',
   'of',
   'the',
   'european',
   'parliament',
   'adjourned',
   'on',
   'friday',
   '17',
   'december',
   '1999,',
   'and',
   'i',
   'would',
   'like',
   'once',
   'again',
   'to',
   'wish',
   'you',
   'a',
   'happy',
   'new',
   'year',
   'in',
   'the',
   'hope',
   'that',
   'you',
   'enjoyed',
   'a',
   'pleasant',
   'festive',
   'period.']),
 (2,
  ['although,',
   'as',
   'you',
   'will',
   'have',
   'seen,',
   'the',
   'dreaded',
   "'millennium",
   "bug'",
   'failed',
   'to',
   'materialise,',
   'still',
   'the',
   'people',
   'in',
   'a',
   'number',
   'of',
   'countries',
   'suffered',
   'a',
   'series',
   'of',
   'natural',
   'disasters',
   'that',
   'truly',
   'were',
   'dreadful.']),
 (3,
  ['you',
   'have',
   'requested',
   'a',
   'debate',
   'on',
   'this',
   'subject',
   'in',
   'the',
   'course

In [90]:
# Join the two RDDs together
sv_en_joined_rdd = sv_swapped.join(en_swapped)

In [91]:
sv_en_joined_rdd.take(10) #check how it looks

24/02/26 21:45:06 INFO SparkContext: Starting job: runJob at PythonRDD.scala:181
24/02/26 21:45:06 INFO DAGScheduler: Registering RDD 124 (join at /tmp/ipykernel_31804/242331830.py:2) as input to shuffle 13
24/02/26 21:45:06 INFO DAGScheduler: Got job 54 (runJob at PythonRDD.scala:181) with 1 output partitions
24/02/26 21:45:06 INFO DAGScheduler: Final stage: ResultStage 74 (runJob at PythonRDD.scala:181)
24/02/26 21:45:06 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 73)
24/02/26 21:45:06 INFO DAGScheduler: Missing parents: List(ShuffleMapStage 73)
24/02/26 21:45:06 INFO DAGScheduler: Submitting ShuffleMapStage 73 (PairwiseRDD[124] at join at /tmp/ipykernel_31804/242331830.py:2), which has no missing parents
24/02/26 21:45:06 INFO MemoryStore: Block broadcast_69 stored as values in memory (estimated size 18.5 KiB, free 433.4 MiB)
24/02/26 21:45:06 INFO MemoryStore: Block broadcast_69_piece0 stored as bytes in memory (estimated size 8.7 KiB, free 433.4 MiB)
24/02/26 2

[(868820,
  (['så',
    'sent',
    'som',
    'januari',
    '2006',
    'började',
    'eu-länderna',
    'inse',
    'att',
    'energi',
    'har',
    'utnyttjats',
    'och',
    'kommer',
    'att',
    'utnyttjas',
    'av',
    'ryssland',
    'som',
    'ett',
    'centralt',
    'utrikespolitiskt',
    'instrument.'],
   ['as',
    'late',
    'as',
    'january',
    '2006,',
    'the',
    'eu',
    'countries',
    'started',
    'to',
    'realise',
    'that',
    'energy',
    'has',
    'been',
    'used',
    'and',
    'will',
    'be',
    'used',
    'by',
    'russia',
    'as',
    'a',
    'central',
    'instrument',
    'of',
    'its',
    'foreign',
    'policies.'])),
 (885305,
  (['på',
    'senare',
    'tid',
    'har',
    'vi',
    'bevittnat',
    'våldsangrepp',
    'på',
    'kristna',
    'präster,',
    'missionärer,',
    'utgivare',
    'och',
    'konvertiter.'],
   ['in',
    'recent',
    'times',
    'we',
    'have',
    'witnessed',
    '

In [92]:
def filter(lines):
    #Filter to exclude empty/missing lines
    filtered_rdd = lines.filter(lambda x: len(x[1][0]) > 1 and len(x[1][1]) > 1)

    #Filter sentences with a small number of words (optional)
    threshold = 5
    filtered_rdd = filtered_rdd.filter(lambda x: len(x[1][0]) < threshold and len(x[1][1]) < threshold)

    # Filter pairs with the same number of words
    filtered_rdd = filtered_rdd.filter(lambda x: len(x[1][0]) == len(x[1][1]))
    return filtered_rdd

In [93]:
filtered_rdd = filter(sv_en_joined_rdd)

In [98]:
filtered_rdd.take(10)

24/02/26 21:50:30 INFO SparkContext: Starting job: runJob at PythonRDD.scala:181
24/02/26 21:50:30 INFO DAGScheduler: Got job 58 (runJob at PythonRDD.scala:181) with 1 output partitions
24/02/26 21:50:30 INFO DAGScheduler: Final stage: ResultStage 82 (runJob at PythonRDD.scala:181)
24/02/26 21:50:30 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 81)
24/02/26 21:50:30 INFO DAGScheduler: Missing parents: List()
24/02/26 21:50:30 INFO DAGScheduler: Submitting ResultStage 82 (PythonRDD[131] at RDD at PythonRDD.scala:53), which has no missing parents
24/02/26 21:50:30 INFO MemoryStore: Block broadcast_74 stored as values in memory (estimated size 14.2 KiB, free 433.4 MiB)
24/02/26 21:50:30 INFO BlockManagerInfo: Removed broadcast_73_piece0 on host-192-168-2-136-de1:10005 in memory (size: 7.8 KiB, free: 434.3 MiB)
24/02/26 21:50:30 INFO BlockManagerInfo: Removed broadcast_73_piece0 on 192.168.2.252:10005 in memory (size: 7.8 KiB, free: 434.3 MiB)
24/02/26 21:50:30 INFO Memor

[(1066660,
  (['kriser', 'skapar', 'alltid', 'möjligheter.'],
   ['crises', 'invariably', 'create', 'opportunities.'])),
 (1159790,
  (['debatten', 'är', 'härmed', 'avslutad.'],
   ['the', 'debate', 'is', 'closed.'])),
 (907160,
  (['-', 'betänkande:', 'christopher', 'heaton-harris'],
   ['-', 'report:', 'christopher', 'heaton-harris'])),
 (1187360, (['(blandade', 'reaktioner)'], ['(mixed', 'reactions)'])),
 (1293000,
  (['frågestunden', 'är', 'härmed', 'avslutad.'],
   ['that', 'concludes', 'question', 'time.'])),
 (1386415, (['bra', 'jobbat!'], ['well', 'done!'])),
 (1712025,
  (['jag', 'förklarar', 'debatten', 'avslutad.'],
   ['the', 'debate', 'is', 'closed.'])),
 (974925,
  (['debatten', 'är', 'härmed', 'avslutad.'],
   ['the', 'debate', 'is', 'closed.'])),
 (1092995,
  (['skriftliga', 'förklaringar', '(artikel', '142)'],
   ['written', 'statements', '(rule', '142)'])),
 (1103760,
  (['debatten', 'är', 'härmed', 'avslutad.'],
   ['the', 'debate', 'is', 'closed.']))]

In [61]:
# Step 4: Filter to exclude empty/missing lines
# filtered_rdd = joined_rdd.filter(lambda x: x[1][0] and x[1][1])

In [17]:

# Step 5: Filter sentences with a small number of words (optional)
# You can add a filter here if needed

# Step 6: Filter pairs with the same number of words
# filtered_rdd = filtered_rdd.filter(lambda x: len(x[1][0]) == len(x[1][1]))

In [95]:
# Step 7: Pair each word in the sentences
word_pairs_rdd = filtered_rdd.map(lambda x: zip(x[1][0], x[1][1]))

In [100]:
print(list(word_pairs_rdd.take(1)[0])) 

24/02/26 21:52:02 INFO SparkContext: Starting job: runJob at PythonRDD.scala:181
24/02/26 21:52:02 INFO DAGScheduler: Got job 60 (runJob at PythonRDD.scala:181) with 1 output partitions
24/02/26 21:52:02 INFO DAGScheduler: Final stage: ResultStage 86 (runJob at PythonRDD.scala:181)
24/02/26 21:52:02 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 85)
24/02/26 21:52:02 INFO DAGScheduler: Missing parents: List()
24/02/26 21:52:02 INFO DAGScheduler: Submitting ResultStage 86 (PythonRDD[133] at RDD at PythonRDD.scala:53), which has no missing parents
24/02/26 21:52:02 INFO MemoryStore: Block broadcast_76 stored as values in memory (estimated size 14.5 KiB, free 433.4 MiB)
24/02/26 21:52:02 INFO BlockManagerInfo: Removed broadcast_75_piece0 on host-192-168-2-136-de1:10005 in memory (size: 7.8 KiB, free: 434.3 MiB)
24/02/26 21:52:02 INFO MemoryStore: Block broadcast_76_piece0 stored as bytes in memory (estimated size 7.8 KiB, free 433.4 MiB)
24/02/26 21:52:02 INFO BlockManage

[('utgör', 'is'), ('detta', 'that'), ('ett', 'a'), ('problem?', 'problem?')]


24/02/26 21:52:21 INFO TaskSetManager: Finished task 0.0 in stage 86.0 (TID 196) in 19410 ms on 192.168.2.250 (executor 1) (1/1)
24/02/26 21:52:21 INFO TaskSchedulerImpl: Removed TaskSet 86.0, whose tasks have all completed, from pool 
24/02/26 21:52:21 INFO DAGScheduler: ResultStage 86 (runJob at PythonRDD.scala:181) finished in 19.424 s
24/02/26 21:52:21 INFO DAGScheduler: Job 60 is finished. Cancelling potential speculative or zombie tasks for this job
24/02/26 21:52:21 INFO TaskSchedulerImpl: Killing all running tasks in stage 86: Stage finished
24/02/26 21:52:21 INFO DAGScheduler: Job 60 finished: runJob at PythonRDD.scala:181, took 19.430921 s


In [102]:
words_rdd = word_pairs_rdd.flatMap(lambda line: line)
word_counts_rdd = words_rdd.map(lambda word: (word, 1))
word_freq_rdd = word_counts_rdd.reduceByKey(lambda a, b: a + b)

# Print 10 of the most frequently occurring pairs of words
top_pairs = word_freq_rdd.takeOrdered(10, key=lambda x: -x[1])
for (word, translation), count in top_pairs:
    print(f'{word} - {translation}: {count}')


24/02/26 22:07:03 INFO SparkContext: Starting job: takeOrdered at /tmp/ipykernel_31804/2986523124.py:6
24/02/26 22:07:03 INFO DAGScheduler: Registering RDD 135 (reduceByKey at /tmp/ipykernel_31804/2986523124.py:3) as input to shuffle 14
24/02/26 22:07:03 INFO DAGScheduler: Got job 61 (takeOrdered at /tmp/ipykernel_31804/2986523124.py:6) with 5 output partitions
24/02/26 22:07:03 INFO DAGScheduler: Final stage: ResultStage 89 (takeOrdered at /tmp/ipykernel_31804/2986523124.py:6)
24/02/26 22:07:03 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 88)
24/02/26 22:07:03 INFO DAGScheduler: Missing parents: List(ShuffleMapStage 88)
24/02/26 22:07:03 INFO DAGScheduler: Submitting ShuffleMapStage 88 (PairwiseRDD[135] at reduceByKey at /tmp/ipykernel_31804/2986523124.py:3), which has no missing parents
24/02/26 22:07:03 INFO MemoryStore: Block broadcast_77 stored as values in memory (estimated size 17.0 KiB, free 433.4 MiB)
24/02/26 22:07:03 INFO BlockManagerInfo: Removed broadcas

avslutad. - closed.: 2534
är - is: 2380
jag - the: 1324
debatten - is: 1324
förklarar - debate: 1317
debatten - the: 1225
härmed - is: 1215
är - debate: 1187
(artikel - (rule: 893
det - that: 852


24/02/26 22:07:51 INFO TaskSetManager: Finished task 4.0 in stage 88.0 (TID 201) in 23361 ms on 192.168.2.252 (executor 0) (5/5)
24/02/26 22:07:51 INFO TaskSchedulerImpl: Removed TaskSet 88.0, whose tasks have all completed, from pool 
24/02/26 22:07:51 INFO DAGScheduler: ShuffleMapStage 88 (reduceByKey at /tmp/ipykernel_31804/2986523124.py:3) finished in 48.170 s
24/02/26 22:07:51 INFO DAGScheduler: looking for newly runnable stages
24/02/26 22:07:51 INFO DAGScheduler: running: Set()
24/02/26 22:07:51 INFO DAGScheduler: waiting: Set(ResultStage 89)
24/02/26 22:07:51 INFO DAGScheduler: failed: Set()
24/02/26 22:07:51 INFO DAGScheduler: Submitting ResultStage 89 (PythonRDD[138] at takeOrdered at /tmp/ipykernel_31804/2986523124.py:6), which has no missing parents
24/02/26 22:07:51 INFO MemoryStore: Block broadcast_78 stored as values in memory (estimated size 11.2 KiB, free 433.4 MiB)
24/02/26 22:07:51 INFO MemoryStore: Block broadcast_78_piece0 stored as bytes in memory (estimated size 

In [20]:


def count_word_frequency(lines):
    #need to add all words in a single array:
    words = lines.flatMap(lambda line: line)   
    word_counts_rdd = words.map(lambda word: (word, 1)) #creating rdd for word counts

    word_freq = word_counts_rdd.reduceByKey(lambda oldcount, increment:  oldcount + increment)
    return word_freq
    

def sort_word_frequency_list(word_freq_rdd):
    # Sort the word counts in descending order
    sorted_word_frequency = word_freq_rdd.sortBy(lambda x: x[1], ascending=False)
    return sorted_word_frequency

orted_words = sort_word_frequency_list(count_word_frequency(word_pairs_rdd))

# Step 8: Use reduceByKey to count occurrences of word-translation pairs
word_translation_counts = word_pairs_rdd.flatMap(lambda x: x).map(lambda x: ((x[0], x[1]), 1)).reduceByKey(lambda a, b: a + b)

# Step 9: Print some of the most frequently occurring pairs of words
top_pairs = word_translation_counts.takeOrdered(10, key=lambda x: -x[1])
for (word, translation), count in top_pairs:
    print(f'{word} - {translation}: {count}')


24/02/26 20:51:08 INFO SparkContext: Starting job: takeOrdered at /tmp/ipykernel_31804/2456824845.py:2
24/02/26 20:51:08 INFO DAGScheduler: Registering RDD 28 (join at /tmp/ipykernel_31804/4098632872.py:6) as input to shuffle 3
24/02/26 20:51:08 INFO DAGScheduler: Registering RDD 32 (reduceByKey at /tmp/ipykernel_31804/3108678048.py:2) as input to shuffle 2
24/02/26 20:51:08 INFO DAGScheduler: Got job 12 (takeOrdered at /tmp/ipykernel_31804/2456824845.py:2) with 5 output partitions
24/02/26 20:51:08 INFO DAGScheduler: Final stage: ResultStage 16 (takeOrdered at /tmp/ipykernel_31804/2456824845.py:2)
24/02/26 20:51:08 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 15)
24/02/26 20:51:08 INFO DAGScheduler: Missing parents: List(ShuffleMapStage 15)
24/02/26 20:51:08 INFO DAGScheduler: Submitting ShuffleMapStage 14 (PairwiseRDD[28] at join at /tmp/ipykernel_31804/4098632872.py:6), which has no missing parents
24/02/26 20:51:08 INFO MemoryStore: Block broadcast_16 stored as v

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 14.0 failed 4 times, most recent failure: Lost task 2.3 in stage 14.0 (TID 41) (192.168.2.252 executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/ubuntu/spark-3.5.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/home/ubuntu/spark-3.5.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 1237, in process
    out_iter = func(split_index, iterator)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pyspark/rdd.py", line 5434, in pipeline_func
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pyspark/rdd.py", line 840, in func
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pyspark/rdd.py", line 4175, in combine
  File "/home/ubuntu/spark-3.5.0-bin-hadoop3/python/lib/pyspark

In [11]:
# release the cores for another application!
# spark_context.stop()

24/02/21 13:10:22 INFO SparkContext: SparkContext is stopping with exitCode 0.
24/02/21 13:10:22 INFO SparkUI: Stopped Spark web UI at http://host-192-168-2-13-de1:4040
24/02/21 13:10:22 INFO StandaloneSchedulerBackend: Shutting down all executors
24/02/21 13:10:22 INFO StandaloneSchedulerBackend$StandaloneDriverEndpoint: Asking each executor to shut down
24/02/21 13:10:22 INFO MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
24/02/21 13:10:22 INFO MemoryStore: MemoryStore cleared
24/02/21 13:10:22 INFO BlockManager: BlockManager stopped
24/02/21 13:10:22 INFO BlockManagerMaster: BlockManagerMaster stopped
24/02/21 13:10:22 INFO OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
24/02/21 13:10:22 INFO SparkContext: Successfully stopped SparkContext
